# Qlunc's presentation

Qlunc stands for **Q**uantification of **l**idar **unc**ertainty. It is stored as a public SWE [repository](https://github.com/SWE-UniStuttgart/Qlunc).

1. Install [anaconda](https://anaconda.org/anaconda/conda)

2. Download or [clone](https://github.com/SWE-UniStuttgart/Qlunc.git) Qlunc´s repository

3. Create an environment and install dependencies

Anaconda prompt:

> conda env create -f environment.yml

> conda activate <environment name> (by default Qlunc_Env)
    

    

## Creating objects as python classes (*Qlunc_Classes.py*)

First we create the lidar components:

In [1]:
# Component 1:
class component1():
    def __init__(self, name, param1, param2, comp1_uncertainty):
        self.name              = name
        self.param1            = param1
        self.param2            = param2
        self.uncertainty       = comp1_uncertainty # python module computing uncertainty of componentA. Defined by the user.

# Component 2:
class component2():
    def __init__(self, name, param1, param2, comp2_uncertainty):
        self.name              = name
        self.param1            = param1
        self.param2            = param2
        self.uncertainty       = comp2_uncertainty # python module computing uncertainty of componentB. Defined by the user.

# Component 3:
class component3():
    def __init__(self, name, param1, param2,param3 ,comp3_uncertainty):
        self.name              = name
        self.param1            = param1
        self.param2            = param2
        self.param3            = param3
        self.uncertainty       = comp3_uncertainty # python module computing uncertainty of componentC. Defined by the user.

Second, we create the lidar modules as a class containing the components defined in the previous step:

In [2]:
# Module 1:
class module1():
    def __init__(self,name,component1,component2,mod1_uncertainty):
        self.name             = name
        self.component1       = Component1
        self.component2       = Component2
        self.uncertainty      = mod1_uncertainty # python module computing uncertainty of module1. Defined by the user.

# Module 2:
class module2():
    def __init__(self,name,component1,mod2_uncertainty):
        self.name             = name
        self.component1       = Component3
        self.uncertainty      = mod2_uncertainty # python module computing uncertainty of module2. Defined by the user. GUM

then we create the lidar class:

In [3]:
# Lidar object:
class lidar():
    def __init__(self, name,module1, module2, lidar_uncertainty):
        self.name        = name
        self.module1     = module1
        self.module2     = module2
        self.uncertainty = lidar_uncertainty # python module computing uncertainty of lidar device. Defined by the user. GUM

## Instantianting process (*Qlunc_Instantiate.py*):
Once we have virtually created the main structure of our lidar object, we instantiate the classes to create the digital objects representing our actual lidar.
All component parameters are inputted through a yaml file (*Qlunc_inputs.yml*). We do this manually here to show the process:
### Components

In [4]:
Component1=component1(name              = 'Comp1',
                      param1            = 'parameter1_Comp1',
                      param2            = 'parameter2_Comp1',
                      comp1_uncertainty = 'unc_func_Comp1')

Component2=component2(name              = 'Comp2',
                      param1            = 'parameter1_Comp2',
                      param2            = 'parameter2_Comp2',
                      comp2_uncertainty = 'unc_func_Comp2')

Component3=component3(name              = 'Comp3',
                      param1            = 'parameter1_Comp3',
                      param2            = 'parameter2_Comp3',
                      param3            = 'parameter3_Comp3',
                      comp3_uncertainty = 'unc_func_Comp3')

In [5]:
print(Component1.name)
print(Component1.param1)
print(Component2.param2)
print(Component3.uncertainty)


Comp1
parameter1_Comp1
parameter2_Comp2
unc_func_Comp3


### Modules

In [6]:
Module1 = module1(name             = 'Mod1',
                  component1       = Component1,
                  component2       = Component2,
                  mod1_uncertainty ='unc_func_Mod1')
Module2 = module2(name             = 'Mod2',
                component1         = Component3,
                mod2_uncertainty   = 'unc_func_Mod2')

In [7]:
print(Module1.component1.param1)
print(Module2.component1.param3)

parameter1_Comp1
parameter3_Comp3


### Lidar

In [8]:
Lidar= lidar(name='Caixa1', module1=Module1,module2=Module2,lidar_uncertainty='Lidar_unc_func')

In [9]:
print(Lidar.uncertainty)

Lidar_unc_func
